In [1]:
import json
from openai import OpenAI

def llm(input: str, is_json_response: bool = False):
    client = OpenAI()

    response = client.responses.create(
        model="gpt-4.1",
        input=input
    )


    if is_json_response:
        response = response.output_text.replace('```', '')
        return json.loads(response)
    
    return response.output_text    

In [2]:
steps = [
  {
    "order": 1,
    "group": "رئيس وحدة المشتريات",
    "content": "مراجعة الاتفاقيات السابقة مع الموردين والتأكد من الالتزام بالشروط قبل إعداد دعوة العطاء.",
    "type": "process"
  },
  {
    "order": 2,
    "group": "أخصائي المشتريات",
    "content": "إعداد جدول المقارنة بين العروض المقدمة من الموردين من حيث السعر، الجودة، ومدة التسليم.",
    "model": "نموذج معتمد",
    "type": "predefined-process"
  },
  {
    "order": 3,
    "group": "مقرر لجنة المشتريات",
    "content": "تنظيم أوراق العمل الخاصة بالاجتماع وتوزيعها على أعضاء اللجنة للاطلاع قبل بدء النقاشات.",
    "type": "process"
  },
  {
    "order": 4,
    "group": "أخصائي المشتريات",
    "content": "التواصل مع الموردين غير المستوفين للشروط وتوضيح أسباب استبعاد عروضهم.",
    "type": "process"
  },
  {
    "order": 5,
    "group": "رئيس وحدة المشتريات",
    "content": "تحديث بيانات الموردين المعتمدين في النظام وفقاً لنتائج التقييم الفني والمالي.",
    "type": "predefined-process"
  },
  {
    "order": 6,
    "group": "مقرر لجنة المشتريات",
    "content": "توثيق ملاحظات اللجنة خلال عملية التقييم الفني ورفعها إلى الجهات المختصة للمراجعة.",
    "model": "نموذج معتمد",
    "type": "process"
  },
  {
    "order": 7,
    "group": "رئيس وحدة المشتريات",
    "content": "تقديم التوصيات للإدارة العليا بناءً على نتائج تقييم الموردين ومدى توافقهم مع متطلبات المشروع.",
    "type": "process"
  },
  {
    "order": 8,
    "group": "أخصائي المشتريات",
    "content": "إعداد مسودة أمر الشراء ومراجعتها مع الفريق القانوني قبل إرسالها للمورد.",
    "model": "نموذج معتمد",
    "type": "predefined-process"
  },
  {
    "order": 9,
    "group": "أخصائي المشتريات",
    "content": "استلام الفواتير من الموردين ومطابقتها مع أوامر الشراء للتأكد من دقة المعلومات.",
    "model": "نموذج معتمد",
    "type": "process"
  },
  {
    "order": 10,
    "group": "رئيس وحدة المشتريات",
    "content": "اعتماد المدفوعات بعد التأكد من استلام الجهة الطالبة للسلع أو الخدمات بشكل كامل.",
    "type": "process"
  },
  {
    "order": 11,
    "group": "مقرر لجنة المشتريات",
    "content": "إعداد تقرير مفصل حول العطاءات غير المقبولة وتقديمه لإدارة المشتريات للنظر فيه.",
    "model": "نموذج معتمد",
    "type": "predefined-process"
  },
  {
    "order": 12,
    "group": "أخصائي المشتريات",
    "content": "تنسيق جداول التسليم مع الموردين ومتابعة تنفيذ البنود الزمنية المحددة في العقد.",
    "type": "process"
  },
  {
    "order": 13,
    "group": "رئيس وحدة المشتريات",
    "content": "أرشفة جميع الوثائق الخاصة بعملية الشراء والاحتفاظ بها للرجوع إليها عند الحاجة.",
    "type": "process"
  }
]

In [3]:
step_order_content = [{
    'order': x['order'],
    'content': x['content']
} for x in steps]

step_groups = [{
    'group': x['group']
} for x in steps]

In [4]:
prompt = """
You are provided with an ordered list of steps required to accomplish a project.

Your task:
Read, analyze, and fully understand the steps and how they relate to one another.
Identify any step that involves decision-making or requires approval to continue.

For each identified decision step, determine:
yes_step: the step order to follow if the decision is approved or true.
no_step: the step order to follow if the decision is rejected or false.

Output Format:
{output}

Important Rules:
{rules}

Your response must be a valid JSON array with no extra text.

Steps:
{steps}
"""

output = """Return only the decision steps in the following valid JSON format:
[
    {{
        "order": step order number that requires a decision or approval,
        "yes_step": <step order if the decision is true>,
        "no_step": <step order if the decision is false>
    }}
]
"""

rules = """
yes_step and no_step must never point to the same step.
yes_step and no_step must never point to the same decision step.
At least one of yes_step or no_step must point to the next step.
"""

In [5]:
prompt = prompt.format(output=output, rules=rules, steps=step_order_content)

In [6]:
result = llm(input=prompt, is_json_response=True)

In [7]:
print(result)

[{'order': 7, 'yes_step': 8, 'no_step': 11}]


In [8]:
result = [{
    **x,
    'type': 'decision'
} for x in result]

In [9]:
print(result)

[{'order': 7, 'yes_step': 8, 'no_step': 11, 'type': 'decision'}]


In [10]:
decision_title_prompt = """
Generate a decision title of 1-2 words of below task.
Return only your answer without any further word

Task:
{task}
"""

step_group_prompt = """
Classify the following task into one of the given groups.
Respond with only the group name — no explanation or extra words.

Task:
{task}

Available Groups:
{groups}
"""

In [11]:
for r in result:
    task = [x['content'] for x in step_order_content if x['order'] == r['order']]
    title = llm(input=decision_title_prompt.format(task=task))
    group = llm(input=step_group_prompt.format(task=task, groups=step_groups))

    r['content'] = title
    r['group'] = group

In [12]:
result

[{'order': 7,
  'yes_step': 8,
  'no_step': 11,
  'type': 'decision',
  'content': 'توصيات الموردين',
  'group': 'رئيس وحدة المشتريات'}]

In [13]:
# Input data
list1 = result
list2 = steps

# Index decision steps by order
decisions_map = {step['order']: step for step in list1}

# Build output with interleaving
steps = []
for step in list2:
    steps.append(step)
    if step['order'] in decisions_map:
        steps.append(decisions_map[step['order']])


In [14]:
steps

[{'order': 1,
  'group': 'رئيس وحدة المشتريات',
  'content': 'مراجعة الاتفاقيات السابقة مع الموردين والتأكد من الالتزام بالشروط قبل إعداد دعوة العطاء.',
  'type': 'process'},
 {'order': 2,
  'group': 'أخصائي المشتريات',
  'content': 'إعداد جدول المقارنة بين العروض المقدمة من الموردين من حيث السعر، الجودة، ومدة التسليم.',
  'model': 'نموذج معتمد',
  'type': 'predefined-process'},
 {'order': 3,
  'group': 'مقرر لجنة المشتريات',
  'content': 'تنظيم أوراق العمل الخاصة بالاجتماع وتوزيعها على أعضاء اللجنة للاطلاع قبل بدء النقاشات.',
  'type': 'process'},
 {'order': 4,
  'group': 'أخصائي المشتريات',
  'content': 'التواصل مع الموردين غير المستوفين للشروط وتوضيح أسباب استبعاد عروضهم.',
  'type': 'process'},
 {'order': 5,
  'group': 'رئيس وحدة المشتريات',
  'content': 'تحديث بيانات الموردين المعتمدين في النظام وفقاً لنتائج التقييم الفني والمالي.',
  'type': 'predefined-process'},
 {'order': 6,
  'group': 'مقرر لجنة المشتريات',
  'content': 'توثيق ملاحظات اللجنة خلال عملية التقييم الفني ورفعها إل

In [15]:
import copy


# Step 1: Map original order to index
original_to_index = {step['order']: [] for step in steps}
for idx, step in enumerate(steps):
    original_to_index[step['order']].append(idx)


# Step 2: Sort steps by original 'order' and give new sequential order
sorted_steps = sorted(steps, key=lambda s: s['order'])
new_steps = []
order_mapping = {}  # Maps original index to new order number

for new_order, step in enumerate(sorted_steps, start=1):
    new_step = copy.deepcopy(step)
    old_order = step['order']
    old_index = original_to_index[old_order].pop(0)
    order_mapping[old_index] = new_order
    new_step['order'] = new_order
    new_steps.append(new_step)


# Step 3: Update yes_step and no_step in decision steps
for idx, step in enumerate(new_steps):
    if step['type'] == 'decision':
        # Find original step index in the original list
        for original_idx, mapped_order in order_mapping.items():
            if mapped_order == step['order']:
                original_decision = steps[original_idx]
                yes_target = original_decision.get('yes_step')
                no_target = original_decision.get('no_step')

                # Find new orders based on old yes/no step orders
                for orig_idx, orig_step in enumerate(steps):
                    if orig_step['order'] == yes_target:
                        step['yes_step'] = order_mapping[orig_idx]
                    if orig_step['order'] == no_target:
                        step['no_step'] = order_mapping[orig_idx]

In [16]:
new_steps

[{'order': 1,
  'group': 'رئيس وحدة المشتريات',
  'content': 'مراجعة الاتفاقيات السابقة مع الموردين والتأكد من الالتزام بالشروط قبل إعداد دعوة العطاء.',
  'type': 'process'},
 {'order': 2,
  'group': 'أخصائي المشتريات',
  'content': 'إعداد جدول المقارنة بين العروض المقدمة من الموردين من حيث السعر، الجودة، ومدة التسليم.',
  'model': 'نموذج معتمد',
  'type': 'predefined-process'},
 {'order': 3,
  'group': 'مقرر لجنة المشتريات',
  'content': 'تنظيم أوراق العمل الخاصة بالاجتماع وتوزيعها على أعضاء اللجنة للاطلاع قبل بدء النقاشات.',
  'type': 'process'},
 {'order': 4,
  'group': 'أخصائي المشتريات',
  'content': 'التواصل مع الموردين غير المستوفين للشروط وتوضيح أسباب استبعاد عروضهم.',
  'type': 'process'},
 {'order': 5,
  'group': 'رئيس وحدة المشتريات',
  'content': 'تحديث بيانات الموردين المعتمدين في النظام وفقاً لنتائج التقييم الفني والمالي.',
  'type': 'predefined-process'},
 {'order': 6,
  'group': 'مقرر لجنة المشتريات',
  'content': 'توثيق ملاحظات اللجنة خلال عملية التقييم الفني ورفعها إل